In [48]:
#pip install xgboost

In [49]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from datetime import datetime
from tensorflow.keras.models import Sequential
from keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, LeakyReLU
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import cross_val_score
from sklearn import datasets
from sklearn.neighbors import KNeighborsClassifier
from sklearn.utils import shuffle
from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.metrics import confusion_matrix
import seaborn as sns

In [50]:
data = pd.read_csv("C:/Users/USER/Desktop/Program/Machine_training/Network_processed_data.csv")
for i in range (10):
    data = shuffle(data)
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)
data.head(10)

,ts,src_ip,src_port,dst_ip,dst_port,proto,service,duration,src_bytes,dst_bytes,conn_state,missed_bytes,src_pkts,src_ip_bytes,dst_pkts,dst_ip_bytes,dns_query,dns_qclass,dns_qtype,dns_rcode,dns_AA,dns_RD,dns_RA,dns_rejected,ssl_version,...,ssl_issuer,http_trans_depth,http_method,http_uri,http_version,http_request_body_len,http_response_body_len,http_status_code,http_user_agent,http_orig_mime_types,http_resp_mime_types,weird_name,weird_addl,weird_notice,label,type_backdoor,type_ddos,type_dos,type_injection,type_mitm,type_normal,type_password,type_ransomware,type_scanning,type_xss
112096,55461,11535,110,5189,0,0,0,77850,192,0,0,0,9,653,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,False,False,False,False,False,True,False,False,False,False
386073,100823,4600,47267,1531,866,1,1,13484,124,2280,10,0,5,294,7,2955,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,False,False,False,False,False,True,False,False,False,False
250091,80109,4595,36497,2772,51,2,1,19779,42,475,10,0,1,35,1,474,1,1,9,0,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,False,False,False,False,False,True,False,False,False,False
284234,84136,4595,14516,4447,51,2,1,64049,50,109,10,0,1,43,1,108,1,1,4,3,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,False,False,False,False,False,True,False,False,False,False
128355,63137,4595,31820,1516,780,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,False,False,False,False,False,True,False,False,False,False
243085,79044,10719,40225,1511,78,1,0,13269,0,0,7,0,1,2,1,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,False,False,False,False,False,True,False,False,False,False
178129,74644,4602,37,1514,40,1,0,117,0,0,1,0,1,2,1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,False,False,True,False,False,False,False,False,False,False
270368,83853,8404,24370,1519,78,1,0,0,0,0,11,0,1,2,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,False,False,False,False,False,True,False,False,False,False
44363,25724,4590,31459,1516,51,2,1,428,0,279,12,0,0,0,2,306,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,False,False,False,False,False,True,False,False,False,False
430787,110175,4608,2505,1518,1131,1,0,0,0,0,0,0,1,215,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,False,False,False,False,False,False,False,True,False,False


In [51]:

#拿出資料90%當作訓練集
par = np.random.rand(len(data)) < 0.9
data_train = data[par]
#剩下的當作test集
data_test = data[~par]
train_data = data.values
test_data = data.values

#前44列的資料進行訓練
X_train = train_data[:,0:44]
X_train =X_train.astype(float)
#第5列之後為目標變量
Y_train = train_data[:,44:]
Y_train =Y_train.astype(float)
#前4列為測試集
X_test = test_data[:,0:44]

#第5列之後為訓練目標變量
Y_test = test_data[:,44:]
Y_test =Y_test.astype(float)


print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)
print("=============")
print(X_train.dtype)
print(Y_test.dtype)

(461043, 44)
(461043, 10)
(461043, 44)
(461043, 10)
float64
float64


In [52]:
# 建立 XGBClassifier 模型
xgboostModel = XGBClassifier(n_estimators=100, learning_rate= 0.3)
# 使用訓練資料訓練模型
xgboostModel.fit(X_train, Y_train)
# 使用訓練資料預測分類
predicted = xgboostModel.predict(X_train)


In [57]:
# 預測成功的比例
# print('訓練集: ',xgboostModel.score(X_train,Y_train))
# print('測試集: ',xgboostModel.score(X_test,Y_test))
print('accuracy: ',xgboostModel.score(X_test,Y_test))

accuracy::  1.0


In [60]:
knn_model = KNeighborsClassifier(n_neighbors = 10, weights='uniform', algorithm='auto', leaf_size=30, p=2, metric='minkowski', metric_params=None, n_jobs=1)
accuracy = cross_val_score(knn_model, X_test, Y_test, cv=10, scoring="accuracy")
print(accuracy)
print('accuracy: ',accuracy.mean()*100,'%')

[0.98631385 0.98514261 0.98670426 0.98713778 0.98694256 0.98681242
 0.98722454 0.98715947 0.98692087 0.98711609]
accuracy:  98.6747444558563 %


In [55]:
# cm=confusion_matrix(Y_test,predicted)
# plt.figure(figsize=(15,8))
# sns.heatmap(cm,square=True,annot=True,fmt='d',linecolor='white',cmap='RdBu',linewidths=1.5,cbar=False)
# plt.xlabel('Pred',fontsize=20)
# plt.ylabel('True',fontsize=20)
# plt.show()